In [15]:
import os
import geopandas as gpd
import pandas as pd
from collections import Counter
from datetime import datetime

In [8]:
file_dir = os.path.join(os.path.expanduser("~"), "Desktop","projects","glims_processor","data")
file_name = "master_glims_df.csv"
df = pd.read_csv(os.path.join(file_dir,file_name), index_col = 0) #read in data

In [9]:
df['date'] = [datetime.strptime(dat.split("T")[0], '%Y-%m-%d') for dat in list(df.src_date)] #convert string dates to datetime
df = df.loc[df['date'] > datetime(2005,1,1)] #select glaciers with polygons drawn after 2005
df = df[~df['geog_area'].isin(['Svalbard, Jan Mayen, and Bouvet', "Various (GlobGlacier)", 'Randolph Glacier Inventory; Umbrella RC for merging the RGI into GLIMS','Antarctic Peninsula','Various (NSIDC)'])]

In [11]:
df = df[(df.db_area > 1) & (df.db_area < 50)]
#create df which is a df of unique glaciers sorted by area, descending
df = df.reset_index(drop = True)
clean_copy = df.copy()
s = set()
for i in range(df.shape[0]):
    if df.iloc[i,0] in s:
        clean_copy = clean_copy.drop(i, axis = 0)
        continue
    else:
        s.add(df.iloc[i,0])
df = clean_copy

In [16]:
ids = list(Counter(df.glac_id).keys()) #get list of unique ids
df_sub = df.sample(n=3000) #get top 3000 largest glaciers
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry = "geometry")
gdf.iloc[:,:-1].to_file("glims_3000_bb.shp")